In [ ]:
# pip 는 설치 하는건데 딥러닝 관련 라이브러리를 설치 하는거
!pip install torchmetrics
!pip install pytorch_lightning

In [ ]:
# 라이브러리 선언
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from pytorch_lightning import LightningModule, Trainer
import torch
import torchmetrics
import pytorch_lightning as pl
from torchmetrics.classification import MulticlassAccuracy

In [ ]:
# pytorch_lightning 을 상속받아서 구현
class LightningModel(pl.LightningModule):
    def __init__(self, model, learning_rate):
        super().__init__()
        self.learning_rate = learning_rate
        self.model = model
        self.train_acc = MulticlassAccuracy(num_classes=10)
        self.valid_acc = MulticlassAccuracy(num_classes=10)
    # 모델이 입력값을 받아서 출력이 나오는 부분
    def forward(self, x):
        return self.model(x)
    # train, val 이 같이 사용하는 코드 부분 
    def _shared_step(self, batch):
        features, true_labels = batch
        logits = self.model(features)
        loss = torch.nn.functional.cross_entropy(logits, true_labels)
        predicted_labels = torch.argmax(logits, dim=1)
        return loss, true_labels, predicted_labels
    # batch 마다 사용이 된다. _step 붙은거는 batch 단위로 실행한다는 뜻이다. 
    def training_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("train_loss", loss)
        self.log("train_acc", self.train_acc(predicted_labels, true_labels), on_epoch=True, on_step=False, prog_bar=True)
        self.model.train()
        return loss  # this is passed to the optimzer for training
    # validation 이 실행되는 코드
    def validation_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
        self.log("valid_loss", loss)
        self.log("valid_acc", self.valid_acc(predicted_labels, true_labels), on_epoch=True, on_step=False, prog_bar=True)
      # test시 실행되는 코드
    def test_step(self, batch, batch_idx):
        loss, true_labels, predicted_labels = self._shared_step(batch)
    # optimizer 를 여기에 설정해야 한다. 
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
        return optimizer

In [ ]:
# 모델의 구조를 짜는 코드
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [ ]:
# 모델을 선언하는 코드
model = Net()
# 모델을 입력으로 받아서 pytorch lighting 형태로 만듬.
model = LightningModel(model, 0.001)
# Train 설정에 필요한 값들을 넣어서 trainer를 생성. 
trainer = Trainer(
    check_val_every_n_epoch=1,
    max_epochs=30,
)

In [ ]:
# 데이터를 불러오는 block
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

# pytorch 에 있는 데이터를 불러옴
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=0)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=0)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# 모델을 돌리는 부분
trainer.fit(model, trainloader)

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
INFO:pytorch_lightning.callbacks.model_summary:
  | Name      | Type               | Params
-------------------------------------------------
0 | model     | Net                | 62.0 K
1 | train_acc | MulticlassAccuracy | 0     
2 | valid_acc | MulticlassAccuracy | 0     
-------------------------------------------------
62.0 K    Trainable params
0         Non-trainable params
62.0 K    Total params
0.248     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.
